### **Content License Agreement**

<font color='red'><b>**WARNING**</b></font> : 본 자료는 삼성청년SW·AI아카데미의 컨텐츠 자산으로, 보안서약서에 의거하여 어떠한 사유로도 임의로 복사, 촬영, 녹음, 복제, 보관, 전송하거나 허가 받지 않은 저장매체를 이용한 보관, 제3자에게 누설, 공개 또는 사용하는 등의 무단 사용 및 불법 배포 시 법적 조치를 받을 수 있습니다.

# 실습 목표 (Objectives)

## 실습 개요

본 실습은 커스텀 데이터셋을 기반으로 **소형 언어 모델(SLM)을 파인튜닝(Fine-tuning)** 하는 전체 워크플로우를 구현하는 것을 목표로 합니다. 이를 통해 데이터 증강(Data Augmentation)부터 모델 경량화 및 특정 태스크 최적화까지의 과정을 직접 경험합니다.

## 실습 진행 목적 및 배경


* **PEFT(LoRA) 기반 모델 최적화**: HuggingFace의 `SmolLM2-135M-Instruct` 모델에 LoRA(Low-Rank Adaptation) 기법을 적용하여, 적은 양의 학습 가능한 파라미터만으로 특정 분류 태스크(simple/complex)에 대한 성능을 극대화하는 방법을 학습합니다.
* **모델 경량화 및 배포 전략 이해**: 대형 모델로 생성한 데이터셋을 기반으로 소형 모델을 파인튜닝함으로써, 실제 서비스 환경에서 효율적으로 동작할 수 있는 경량 모델을 구축하고 배포하는 전략을 탐구합니다.
* **엔드투엔드(End-to-End) 파이프라인 구현**: 데이터 생성, 전처리, 모델 훈련, 평가, 그리고 배포용 모델 저장까지의 전 과정을 직접 구현하며 실무적인 MLOps 역량을 강화합니다.

## 실습 수행으로 얻어갈 수 있는 역량

* **PEFT(Parameter-Efficient Fine-Tuning) 기술**: LoRA를 활용하여 전체 파라미터를 재학습하지 않고도 특정 태스크에 맞게 모델을 효율적으로 튜닝하는 기술.
* **HuggingFace 및 PyTorch 활용 능력**: `transformers`, `datasets`, `peft` 등 최신 라이브러리와 PyTorch를 활용하여 모델 훈련 파이프라인을 처음부터 구현하는 능력.

## 실습 핵심 내용


1.  **모델 및 토크나이저 준비**: `HuggingFaceTB/SmolLM2-135M-Instruct` 모델과 토크나이저를 불러와 시퀀스 분류(Sequence Classification) 태스크에 맞게 설정.
2.  **LoRA 적용 및 파인튜닝**: 베이스 모델에 LoRA 설정을 적용하여 학습 가능한 파라미터 수를 최소화하고, 생성된 데이터셋으로 모델을 파인튜닝하여 'suggested_model' (small/large)을 분류하도록 학습.
3.  **모델 저장 및 배포 준비**: 학습된 LoRA 어댑터(Adapter)를 저장하고, 필요시 베이스 모델과 병합(merge)하여 추론(Inference)에 바로 사용할 수 있는 형태로 모델을 저장.

# 5-2 챕터의 흐름

5-2 챕터: 타겟 디바이스용 모델 준비 및 변환

이번 챕터에서는 모델을 타겟 디바이스에 배포할 수 있도록 준비하고, 특정 형식으로 변환하는 과정을 다룹니다.

먼저 5-2-1에서는 라우팅 제어에 사용할 플래그를 만들기 위해 데이터셋을 생성하고, 이를 활용해 소형 언어 모델(SLM)을 미세 조정(Fine-Tuning)합니다.

이어서 5-2-2에서는 준비된 PyTorch 모델을 TFLite라는 중간 형식으로 변환하는 과정을 학습합니다. 이 과정에서 정적 그래프 변환의 장점을 살펴봅니다. 더 나아가 이 챕터의 핵심은 '왜 최종 변환이 한 번 더 필요한가'를 이해하는 것입니다.

각 디바이스의 신경망 실행 엔진은 단순히 언어만 바꾼 것이 아니라, 하드웨어 구조에 맞춰 완전히 별개로 개발된 프레임워크입니다. 
따라서 자연스레 모든 디바이스에서 작동하는 소스코드는 존재하지 않으며, 각 프레임워크로의 변환 자체만으로도 별도의 긴 태스크가 되는 경우가 많습니다. 
TFLite로의 변환은 많은 경우 그 최종 변환의 준비가 되며, 최종 변환의 준비과정만으로 5-2-2와 같은 과정이 필요하게 됩니다. 
해당 과정의 길이와 난이도 상, 이번 3주간의 정규 과정에서 다루기에는 한계가 있어 중간 단계인 TFLite에서 과정을 마무리하게 됩니다. 
이러한 과정이 있다는 사실을 아는 것을 통해 TFLite 파일을 최종적으로 디바이스별 프레임워크로 다시 변환해야만 하는 이유를 명확히 이해하게 됩니다. 

차후 확장적으로 목적하는 장치에 맞춰 변환 혹은 LLM 활용을 진행하시기 위해서는, 아래의 링크를 참고해 주세요. 

타겟 디바이스 별 프레임워크 추가 참고 자료 : 

Qualcomm - https://docs.qualcomm.com/bundle/publicresource/topics/80-63442-100/how_to_use_genie.html?product=1601111740062489 \
Android - https://ai.google.dev/edge/mediapipe/solutions/genai/llm_inference/index?hl=ko \
Raspberry Pi : Hailo(NPU) - https://hailo.ai/blog/bringing-generative-ai-to-the-edge-llm-on-hailo-10h/ \
Apple - https://machinelearning.apple.com/research/core-ml-on-device-llama \
Arduino - https://docs.m5stack.com/en/stackflow/overview \
BeagleBone - https://docs.beagleboard.org/boards/beaglebone/ai-64/index.html \
Radxa / Rockchips - https://docs.radxa.com/en/rock5/rock5b/app-development/rkllm_install




In [ ]:
%pip install -qU langchain
%pip install koreanize-matplotlib
%pip install datasets
%pip install trl
%pip install evaluate




필요 패키지



langchain  ==  0.3.27

pymupdf  ==  1.26.3

datasets  ==  4.0.0


-----

## 실습 개요 (Assignment Overview)

### 들어가며: 커스텀 데이터셋에 대한 소형 모델 최적화하기

최근 대규모 언어 모델(LLM)의 발전으로 고품질의 텍스트 데이터를 프로그래밍 방식으로 생성하는 것이 가능해졌습니다. 이러한 **합성 데이터셋(Synthetic Dataset)**은 특정 태스크에 맞는 학습 데이터를 구하기 어려운 경우 매우 효과적인 대안이 됩니다.

본 실습에서는 생성된 데이터로 특정 목적에 맞게 경량화된 **소형 언어 모델(SLM)**을 **PEFT(Parameter-Efficient Fine-Tuning)** 방식으로 튜닝하는 엔드투엔드(End-to-End) 파이프라인을 구현합니다.

### 실습 목차 (Table of Contents)

이 노트북은 세 가지 주요 파트로 구성되어 있습니다. 사전 준비에 해당하는 부분의 스크립트는 미리 제공해드리오니, 읽어보시며 진행해주세요.


1.  **데이터셋 전처리 및 준비**

      * 생성된 데이터를 `pandas`와 `datasets` 라이브러리를 이용해 불러오고, 모델 학습에 적합한 형태로 변환합니다. 전체 데이터를 학습(Train), 검증(Validation), 테스트(Test)용으로 분할합니다.

2.  **PEFT(LoRA)를 적용한 모델 미세조정**

      * `HuggingFaceTB/SmolLM2-135M-Instruct` 모델을 기반으로, LoRA(Low-Rank Adaptation) 기법을 적용하여 적은 비용으로 효율적인 미세조정을 진행합니다. 문장의 복잡도에 따라 적절한 모델(small/large)을 추천하도록 분류 모델을 학습시킵니다.

3.  **모델 저장 및 배포 준비**

      * 학습이 완료된 모델의 LoRA 가중치(Adapter)를 저장합니다. 필요시, 베이스 모델과 가중치를 병합(merge)하여 추론 환경에 바로 배포할 수 있는 형태로 만듭니다.

### 커스텀 데이터셋 확인
생성된 CSV 파일을 pandas DataFrame으로 로드하여 상위 10개 행을 출력하고, 총 데이터 수를 확인합니다. 이를 통해 데이터가 의도한 형식과 내용으로 정상적으로 생성되었는지 검증합니다. 또한, JSONL 파일의 내용도 일부 출력하여 형식을 확인합니다.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# 셀 6: 생성된 파일을 로컬로 다운로드
from google.colab import files
import os
import shutil


# Google Drive에 필요 데이터셋 저장

# Google Drive가 마운트되어 있어야 합니다. (이전 셀에서 마운트)
drive_dataset_path = "/content/drive/MyDrive/intent_classification_dataset"
os.makedirs(drive_dataset_path, exist_ok=True)

# 파일 복사
csv_path = "intent_dataset.csv"
jsonl_path = "intent_dataset.jsonl"

if os.path.exists(csv_path):
    shutil.copy(csv_path, drive_dataset_path)
    print(f"Copied {csv_path} to {drive_dataset_path}")

if os.path.exists(jsonl_path):
    shutil.copy(jsonl_path, drive_dataset_path)
    print(f"Copied {jsonl_path} to {drive_dataset_path}")

print(f"Dataset files saved to Google Drive: {drive_dataset_path}")

In [ ]:
# 셀 5: 생성된 CSV/JSONL 파일을 확인하고 간단히 미리보기
import pandas as pd
csv_path = "intent_dataset.csv"
jsonl_path = "intent_dataset.jsonl"


print("CSV 존재 여부:", bool(pd.io.common.file_exists(csv_path)))
if pd.io.common.file_exists(csv_path):
    df = pd.read_csv(csv_path)
    display(df.head(10))
    print("총 행 수:", len(df))

# JSONL도 확인
if pd.io.common.file_exists(jsonl_path):
    with open(jsonl_path, "r", encoding="utf-8") as f:
        lines = f.readlines()
    print("JSONL 샘플 (최초 5줄):")
    print("".join(lines[:5]))
    print("총 JSONL 라인 수:", len(lines))

# 01. 생성 데이터셋을 활용한 PEFT 활용 모델 Fine-Tuning
이번 세션에서 사용할 모델은 Smollm2-135m으로, 본래 SequenceClassification을 위한 가중치가 존재하지 않습니다. 이에 이렇게 불러오는 모델의 끝단은 초기화된 가중치를 가진 Score 레이어가 존재하게 되며, 이번 훈련에서 해당 레이어를 포함한 다른 Linear 레이어들을 LoRA를 활용해 훈련합니다.

### 모델 훈련을 위한 라이브러리 임포트
모델 미세조정(Fine-tuning)에 필요한 라이브러리들을 임포트합니다. sklearn은 데이터 분할, torch는 딥러닝 모델의 기반, transformers는 사전 학습된 모델과 토크나이저를 로드하기 위해 사용됩니다. peft 라이브러리는 LoRA와 같은 파라미터 효율적 미세조정 기법을 적용하기 위해 필요합니다. 재현성을 위해 시드(SEED) 값을 고정합니다.

In [ ]:
# 셀 2: 기본 임포트 및 설정
import os
import random
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import torch
from datasets import Dataset, DatasetDict

from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding,
    TrainerCallback,
)

# 재현성
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
os.environ["PYTHONHASHSEED"] = str(SEED)

model_name = "HuggingFaceTB/SmolLM2-135M-Instruct"

In [ ]:
import os, random, numpy as np, pandas as pd
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding
from peft import LoraConfig, get_peft_model, PeftModel

# reproducibility
SEED = 42
random.seed(SEED); np.random.seed(SEED); os.environ["PYTHONHASHSEED"]=str(SEED)

# 유틸: 모델의 모듈 이름들을 출력/검색해서 LoRA 타겟 후보를 추천
def list_module_names(model, max_items=200):
    names = []
    for n, m in model.named_modules():
        names.append(n)
    # 상위 일부만 출력
    print("=== first 200 module names (truncated) ===")
    for i, nm in enumerate(names[:max_items]):
        print(i, nm)
    return names

def suggest_target_modules_from_names(names):
    # 흔히 사용하는 키워드 기반 추천
    candidates = set()
    keywords = ["q_proj","k_proj","v_proj","o_proj","q","k","v","gate","down_proj","up_proj","dense","proj","wq","wk","wv","wo","attn"]
    for nm in names:
        for kw in keywords:
            if kw in nm:
                candidates.add(nm.split('.')[-1])  # 마지막 파트 기준으로 제안
    # 정렬/중복 제거
    return sorted(list(candidates))[:20]


### 학습 데이터 로드 및 레이블 인코딩
앞서 생성한 intent_dataset.csv 파일을 pandas DataFrame으로 로드합니다. 훈련 태스크의 목표 변수(target variable)인 suggested_model 컬럼의 텍스트 레이블('small', 'large')을 모델이 이해할 수 있도록 숫자(0, 1)로 변환하는 label2id 및 id2label 맵을 정의합니다.



만일 위에서 잘못된 패스에 데이터셋을 올려 데이터셋 존재 여부가 False가 뜨신 분이라면, 여기서 Upload 하시면 됩니다. 


In [ ]:
# 필요시 업로드할 수 있게 처리
csv_path = "intent_dataset.csv"
if not os.path.exists(csv_path):
    from google.colab import files
    print("upload intent_dataset.csv (or similarly named file)")
    uploaded = files.upload()
    csv_path = list(uploaded.keys())[0]

df = pd.read_csv(csv_path)
print("loaded", len(df), "rows; sample:")
display(df.head())
# label encoding
label2id = {"small": 0, "large": 1}
id2label = {0: "small", 1: "large"}



### 데이터셋 전처리 및 PyTorch DataLoader 생성

모델 학습을 위한 데이터 준비 과정을 수행합니다. 이 단계는 다음의 하위 과정으로 구성됩니다:

PyTorch Dataset 클래스 정의: torch.utils.data.Dataset을 상속받아 커스텀 IntentDataset 클래스를 정의합니다. 이 클래스는 데이터를 토큰화하고 모델 입력 형식에 맞는 텐서(tensor)로 변환하는 역할을 합니다.

데이터 분할: 전체 데이터셋을 sklearn의 train_test_split을 사용하여 학습(train), 검증(validation), 테스트(test) 세트로 8:1:1 비율로 계층적 분할(stratified split)합니다.

데이터셋 및 DataLoader 인스턴스화: 분할된 각 데이터셋에 대해 IntentDataset과 DataLoader를 생성합니다. DataLoader는 배치(batch) 단위로 데이터를 모델에 효율적으로 공급하는 역할을 합니다.

In [ ]:
# [문제]: PyTorch의 Dataset 클래스를 상속받아 커스텀 데이터셋을 완성합니다.
class IntentDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=128):
        # 1. 레이블 데이터를 텐서로 변환하여 저장합니다.
        self.labels = torch.tensor(dataframe['label_id'].values, dtype=torch.long)

        # [문제 1]: __init__ 메서드에서 데이터프레임의 'text' 컬럼 전체를 한 번에 토크나이징하여
        # self.encodings에 저장합니다. 이는 학습 전, 미리 토크나이징을 진행하여 효율을 크게 높이는 중요한 처리 방식입니다.
        # truncation, padding, max_length, return_tensors='pt' 옵션을 사용해야 합니다.
        # [START CODE]
        self.encodings = tokenizer(
            list(??),
            truncation=??,
            padding=??,
            max_length=??,
            return_tensors=??
        )
        # [END CODE]

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        # [문제 2]: __getitem__ 메서드에서는 미리 토크나이징된 self.encodings에서
        # idx에 해당하는 데이터를 가져와야 합니다.
        # 올바른 텐서 조각(slice)을 가져와 'labels' 키에 레이블을 추가하여 딕셔너리 형태로 반환하세요.
        # [START CODE]
        item = {key: val[??] for key, val in self.encodings.items()}
        item['labels'] = self.labels[??]
        # [END CODE]
        return item

### 사전 학습된 모델 및 토크나이저 로드
HuggingFace Hub로부터 HuggingFaceTB/SmolLM2-135M-Instruct 모델을 로드합니다. 이 모델은 시퀀스 분류(Sequence Classification) 태스크를 수행할 수 있도록 AutoModelForSequenceClassification 클래스를 사용하여 초기화하며, num_labels와 레이블 맵(id2label, label2id)을 지정합니다. 해당 모델에 맞는 토크나이저 또한 AutoTokenizer를 통해 로드합니다.

In [ ]:
# train/val/test split (80/10/10)
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader


SEED = 42

# ---------------------------
# 1. Binary 라벨 매핑
# ---------------------------
df['label_id'] = df['suggested_model'].map({'small': 0, 'large': 1})

# ---------------------------
# [문제 3]: 전체 데이터프레임(df)을 학습+검증(train_val) 데이터와 테스트(test_df) 데이터로 9:1 비율로 분할합니다.
# stratify 옵션을 사용하여 레이블 비율을 유지해야 합니다.
# ---------------------------
# [START CODE]
train_val, test_df = train_test_split(
    ??, test_size=??, stratify=??, random_state=SEED
)
# [END CODE]

# ---------------------------
# [문제 4]: 위에서 나눈 학습+검증(train_val) 데이터를 다시 학습(train_df)과 검증(val_df) 데이터로 분할합니다.
# 학습 데이터의 10%가 검증 데이터가 되도록 test_size를 조절해야 합니다.
# ---------------------------
# [START CODE]
train_df, val_df = train_test_split(
    ??, test_size=??, stratify=??, random_state=SEED
)
# [END CODE]

# ---------------------------
# 3. Tokenizer
# ---------------------------
tokenizer = AutoTokenizer.from_pretrained(model_name)
max_length = 128

# ---------------------------
# 4. Dataset
# ---------------------------
train_ds = IntentDataset(train_df, tokenizer, max_length=max_length)
val_ds   = IntentDataset(val_df, tokenizer, max_length=max_length)
test_ds  = IntentDataset(test_df, tokenizer, max_length=max_length)

print("train/val/test:", len(train_ds), len(val_ds), len(test_ds))

# ---------------------------
# 5. DataLoader 연결
# ---------------------------
train_loader = DataLoader(train_ds, batch_size=16, shuffle=True)
val_loader   = DataLoader(val_ds, batch_size=16)
test_loader  = DataLoader(test_ds, batch_size=16)

### LoRA(Low-Rank Adaptation) 설정 적용
파라미터 효율적 미세조정(PEFT) 기법인 LoRA를 모델에 적용합니다. LoraConfig를 통해 LoRA의 주요 하이퍼파라미터(r, lora_alpha, target_modules 등)를 설정합니다. get_peft_model 함수를 사용하여 기본 모델(base_model)에 LoRA 설정을 적용한 PEFT 모델을 생성합니다. print_trainable_parameters를 통해 전체 파라미터 대비 학습 대상 파라미터의 비율을 확인하여 LoRA가 얼마나 효율적인지 검증합니다.

In [ ]:
MODEL_NAME = "HuggingFaceTB/SmolLM2-135M-Instruct"  # 또는 사용하실 정확한 허브 네임 (변경 가능)
# 일부 사용자 업로드/비공식 model은 trust_remote_code=True 가 필요할 수 있음
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)

# [문제 5]: HuggingFace의 AutoModelForSequenceClassification을 사용하여 사전 학습된 모델을 로드합니다.
# num_labels, id2label, label2id를 파라미터로 전달하여 분류 태스크에 맞게 모델을 초기화해야 합니다.
# [START CODE]
base_model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=??,
    id2label=??,
    label2id=??,
    trust_remote_code=True
)
# [END CODE]
print("Loaded AutoModelForSequenceClassification successfully.")


# [문제 6]: LoRA 설정을 위한 LoraConfig를 정의합니다.
# r=8, lora_alpha=32, 타겟으로는 "all-linear"를 설정하고, 태스크 유형을 시퀀스 분류("SEQ_CLS")로 지정합니다.
# [START CODE]
lora_config = LoraConfig(
    r=??,
    lora_alpha=??,
    target_modules=??,
    lora_dropout=0.05,
    bias="none",
    task_type=??
)
# [END CODE]

model = get_peft_model(base_model, lora_config)
model.print_trainable_parameters()  # 학습 가능한 파라미터 요약 확인

### 모델 학습 실행
PyTorch를 사용한 표준적인 학습 루프(training loop)를 구현하여 모델을 학습시킵니다.

준비: 모델을 GPU로 이동시키고, AdamW 옵티마이저와 CrossEntropyLoss 손실 함수를 정의합니다.

학습 루프: 지정된 에포크(epoch) 수만큼 반복하며, 각 에포크마다 train_loader로부터 배치 단위로 데이터를 받아 순전파(forward pass), 손실 계산, 역전파(backward pass), 옵티마이저 스텝을 실행합니다.

검증 루프: 각 에포크의 학습이 끝난 후, val_loader를 사용하여 검증 데이터셋에 대한 모델의 성능(손실 및 정확도)을 평가합니다. 이를 통해 과적합(overfitting) 여부를 모니터링합니다.

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer
from sklearn.model_selection import train_test_split
from tqdm import tqdm

# ---------------------------
# 1. label 처리: small=0, large=1
# ---------------------------
df['label_id'] = df['suggested_model'].map({'small': 0, 'large': 1})
print(df[['suggested_model','label_id']].head(10))


# ---------------------------
# 2. Train/Val/Test split
# ---------------------------
SEED = 42
train_val, test_df = train_test_split(df, test_size=0.1, stratify=df['label_id'], random_state=SEED)
train_df, val_df = train_test_split(train_val, test_size=0.1111111, stratify=train_val['label_id'], random_state=SEED)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
max_length = 128

train_ds = IntentDataset(train_df, tokenizer, max_length=max_length)
val_ds   = IntentDataset(val_df, tokenizer, max_length=max_length)
test_ds  = IntentDataset(test_df, tokenizer, max_length=max_length)

train_loader = DataLoader(train_ds, batch_size=16, shuffle=True)
val_loader   = DataLoader(val_ds, batch_size=16)
test_loader  = DataLoader(test_ds, batch_size=16)

# ---------------------------
# 4. Model + optimizer
# ---------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.train()

optimizer = torch.optim.AdamW(model.parameters(), lr=2e-4)
criterion = torch.nn.CrossEntropyLoss()  # Binary classification도 CrossEntropyLoss 사용 가능

# ---------------------------
# 5. Training Loop
# ---------------------------
num_epochs = 5
for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")
    epoch_loss = 0
    correct = 0
    total = 0

    for batch in tqdm(train_loader):
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        # [문제 7]: 모델의 순전파, 손실 계산, 역전파 과정을 완성합니다.
        # 1. 모델에 input_ids와 attention_mask를 전달하여 출력을 얻습니다.
        # 2. CrossEntropyLoss를 사용하여 모델의 출력(logits)과 실제 레이블(labels) 간의 손실을 계산합니다.
        # 3. 계산된 손실에 대해 역전파를 수행합니다.
        # 4. 옵티마이저를 사용하여 모델의 가중치를 업데이트합니다.
        # [START CODE]
        outputs = model(input_ids=??, attention_mask=??)
        logits = outputs.logits
        loss = criterion(??, ??)
        ??.backward()
        ??.step()
        # [END CODE]

        epoch_loss += loss.item() * input_ids.size(0)
        preds = logits.argmax(dim=-1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    print(f"Train loss: {epoch_loss/total:.4f}, accuracy: {correct/total:.4f}")

    # ---------------------------
    # Validation
    # ---------------------------
    model.eval()
    val_loss = 0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            loss = criterion(logits, labels)

            val_loss += loss.item() * input_ids.size(0)
            preds = logits.argmax(dim=-1)
            val_correct += (preds == labels).sum().item()
            val_total += labels.size(0)

    print(f"Val   loss: {val_loss/val_total:.4f}, accuracy: {val_correct/val_total:.4f}")
    model.train()


### 학습된 모델 가중치 저장
학습이 완료된 모델의 가중치를 저장하여 추후 재사용 및 배포가 가능하도록 합니다. 이 셀에서는 Google Drive를 마운트하여 Colab 세션이 종료된 후에도 파일이 유지되도록 합니다.

LoRA 어댑터 저장: save_pretrained 메소드를 사용하여 학습된 LoRA 가중치(어댑터)만 별도로 저장합니다. 이는 용량이 작아 관리가 용이합니다.

모델 병합 및 저장: PeftModel을 사용하여 기본 모델에 학습된 LoRA 어댑터를 병합(merge)합니다. 병합된 전체 모델은 추론(inference) 시 추가적인 처리 없이 바로 사용할 수 있는 상태가 되며, 이 모델을 Google Drive에 저장하여 다음 모듈인 모듈 14, 5-2-2에서 사용합니다.

In [ ]:
import os

# adapter만 저장
peft_save_dir = "peft_smollm2_adapters"
model.save_pretrained(peft_save_dir)
print("Saved LoRA adapters to:", peft_save_dir)

# Define the path to save the model in Google Drive
drive_save_path = "/content/drive/MyDrive/smollm2_merged_for_inference"

# Create the directory in Google Drive if it doesn't exist
os.makedirs(drive_save_path, exist_ok=True)


# [문제 8]: 학습된 LoRA 어댑터를 기본 모델과 병합하여 배포용 모델을 생성합니다.
# merge_and_unload() 함수를 사용하세요.
# [START CODE]
# 배포/인퍼런스용으로 base + adapter 합치기
merged = PeftModel.from_pretrained(base_model, peft_save_dir)
merged_model = merged.??
# [END CODE]

# Save the merged model and tokenizer to Google Drive
merged_model.save_pretrained(drive_save_path)
tokenizer.save_pretrained(drive_save_path)
print(f"Merged model saved to {drive_save_path}")

### 분류 헤드(Classification Head) 가중치 저장
미세조정 과정에서 학습된 부분 중, 최종 분류를 담당하는 score 레이어(분류 헤드)의 가중치만 별도로 저장합니다. 이는 모델의 특정 부분만 분석하거나 다른 모델에 이식할 때 유용할 수 있습니다.

In [ ]:
torch.save(merged_model.score.state_dict(), 'score_only.pt')
merged_model.score

### 저장된 모델을 이용한 추론(Inference) 테스트
저장된 병합 모델을 다시 불러와 실제 데이터에 대한 예측 성능을 테스트합니다.

모델 로드: Google Drive에 저장된 모델과 토크나이저를 로드합니다. 로딩 실패 시, 기본 모델에 어댑터를 다시 적용하는 대체 로직이 포함되어 있습니다.

예측 함수 정의: 텍스트를 입력받아 토큰화하고, 모델을 통해 예측된 레이블('small' 또는 'large')을 반환하는 predict_suggested_model 함수를 정의합니다.

성능 검증: 사전에 정의된 테스트 문장들에 대해 예측을 수행하고, 예측 결과와 정답을 비교하여 모델이 새로운 데이터에 대해 얼마나 잘 일반화하는지 확인합니다.

In [ ]:
# 셀 8: 모델 추론 예시
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from peft import PeftModel

# 저장된 모델 경로
drive_save_path = "/content/drive/MyDrive/smollm2_merged_for_inference"

# 저장된 merged model과 tokenizer 로드
try:
    inference_model = AutoModelForSequenceClassification.from_pretrained(drive_save_path)
    inference_tokenizer = AutoTokenizer.from_pretrained(drive_save_path)
    print("Merged model loaded successfully for inference.")
except Exception as e:
    print(f"Failed to load merged model from {drive_save_path}: {e}")
    print("LoRA adapter와 base model을 따로 로드하여 테스트합니다.")
    # 만약 merged model 로딩에 실패하면, base model에 adapter를 연결하여 사용
    base_model_path = "HuggingFaceTB/SmolLM2-135M-Instruct" # base model 경로
    base_model_inf = AutoModelForSequenceClassification.from_pretrained(
        base_model_path,
        num_labels=2,
        id2label=id2label, # 이전 셀에서 정의된 id2label 사용
        label2id=label2id, # 이전 셀에서 정의된 label2id 사용
        trust_remote_code=True
    )
    peft_save_dir = "peft_smollm2_adapters" # adapter 경로 (이전 셀에서 저장한 경로)
    inference_model = PeftModel.from_pretrained(base_model_inf, peft_save_dir)
    inference_model = inference_model.merge_and_unload() # 다시 merge 시도
    inference_tokenizer = AutoTokenizer.from_pretrained(base_model_path, trust_remote_code=True)
    print("Loaded base model and adapter, then merged for inference.")


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
inference_model.to(device)
inference_model.eval() # 평가 모드

# 예측 함수
def predict_suggested_model(text):
    inputs = inference_tokenizer(
        text,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=128
    ).to(device)

    # [문제 9]: 모델 추론 과정을 완성합니다.
    # 1. torch.no_grad() 컨텍스트 내에서 모델의 순전파를 수행하여 출력을 얻습니다.
    # 2. 출력(logits)에서 가장 높은 값을 가진 인덱스를 찾아 예측 결과를 생성합니다.
    # [START CODE]
    with torch.no_grad():
        outputs = inference_model(??)
        logits = outputs.logits
        predictions = torch.argmax(??, dim=-1)
    # [END CODE]

    predicted_id = predictions.item()
    predicted_label = inference_model.config.id2label[predicted_id]
    return predicted_label

# 예시 테스트와 정답 비교
# 복잡도 및 suggested_model 정보는 예시 리스트에 직접 명시합니다.
test_sentences_with_info = [
    {"text": "회원가입은 어떻게 하나요?", "complexity": "simple", "correct_model": "small"}, # simple account
    {"text": "주문한 상품의 배송 상태를 추적하고 싶습니다.", "complexity": "simple", "correct_model": "small"}, # simple delivery
    {"text": "결제 시 사용 가능한 할인 쿠폰이 있나요?", "complexity": "simple", "correct_model": "large"}, # simple order (DEFAULT_ROUTER['주문_결제'] == False)
    {"text": "로그인 시도 시 오류 코드가 발생하는데 해결 방법은 무엇인가요?", "complexity": "complex", "correct_model": "large"}, # complex tech support
    {"text": "주문 후 결제 수단을 변경할 수 있나요?", "complexity": "complex", "correct_model": "large"}, # complex order
    {"text": "이 제품의 상세 스펙과 사용 후기를 알고 싶습니다.", "complexity": "complex", "correct_model": "large"} # complex product info
]

print("\n--- Test Predictions ---")
for item in test_sentences_with_info:
    sentence = item['text']
    complexity = item['complexity']
    correct_label = item['correct_model']

    predicted_model = predict_suggested_model(sentence)

    is_correct = "Correct" if predicted_model == correct_label else "Incorrect"

    print(f"'{sentence}'")
    print(f"  -> Complexity: {complexity}, Correct Model: {correct_label}, Predicted: {predicted_model}, Result: {is_correct}")

# 마치며 (Conclusion)



본 실습에서 해당 분류 태스크는, 높아야 약 90퍼센트의 정확도만을 얻게 됩니다.
더 정확도를 높이기 위해서는 어떤 조건이 필요할까요?


학습 시간을 더 늘린다고 해서 성능이 눈에 띄게 오를 확률은 높지 않습니다. 하지만 걱정하지 않으셔도 좋습니다. 이번 태스크는 애초에 현재 크기의 모델과 데이터셋으로 100%의 정확도를 달성하는 것을 목표로 삼지 않기 때문입니다.


본 실습에 사용된 데이터셋은 본 실습의 "전체 코드" 가 되는, 곧 만나실 과제에서 가져온 것으로, 'DEFAULT_ROUTER'라는 설정을 통해 "특정 주제는 의도적으로 큰 모델에 매핑한다" 라는 규칙이 들어가도록 만들어졌습니다.


정리하자면, 이 태스크는 문장 자체의 복잡성 외에도, 질문이 가진 특정 주제에 따라 매핑 규칙을 수동으로 적용하여 개발자의 의도를 반영하도록 만든 것입니다. 그러나 이렇게 라벨링을 변형하면 추론의 난이도가 상당히 높아집니다.


예시를 들어보겠습니다. 극단적으로 단순한 데이터로서, 모든 데이터가 알파벳 A, 알파벳 B 하나씩으로만 이루어져있고 그 라벨이 A면 0, B면 1로 하도록 설정했다고 생각해봅시다. 이때 A의 라벨 50%를 1로 변경한 뒤 학습한다면, 어떻게 될까요?


이런 설정에서 모델은, 두 A간의 차이를 알 수 없기에 라벨이 0인 A와 라벨이 1인 A를 구분할 수 없습니다.  따라서 모델은 A에 대해선 주어진 라벨과 어떤 연관성을 얻어갈 수 없게 됩니다. 따라서 모델의 학습이 불가능합니다. 


그러나 지금의 경우는 학습이 불가능하지 않습니다. 데이터 자체가 자연어 문장형으로 이루어져, 주제에 대한 "문맥/Context"이 데이터에 들어있기 때문입니다. 


**그리고 문맥은 자연어 Task에서 가장 강력한 키워드입니다.**


문장의 복잡도가 아닌 주제로 인해 특정 모델을 배정받게 되는 질의에서 자주 사용되는 단어, 표현, 문장 구조 등으로 인해, 모델은 해당 질의에 대해서도 라벨과의 상관관계를 구축할 수 있게 됩니다. 따라서 모델이 충분히 컸다면, 그리고 데이터가 충분히 많았다면 지금의 설정에서도 더 높은 정확도를 얻을 수 있었을 것입니다. 


**하지만, 더 큰 모델과 더 큰 데이터셋만이 유일한 방법일까요?**


---


### 더 나아가기: 문제를 쪼개서 생각하기

혹시 이 문제를 조금 더 단순하게 만들어, 모델이 배우기 쉽게 할 방법은 없었을까요?  예시를 들어보겠습니다. 여기서 중요한 개념은 바로 태스크 설계(Task Design)입니다.

지금의 태스크는 모델에게 단 하나의 질문을 던지는 태스크입니다.


**‘이 입력을 일반 모델로 보낼까, 대형 모델로 보낼까?’**


언뜻 보면 단순해 보이지만, 사실 이 안에는 ‘주제가 큰 모델을 요구하는가’, ‘문장이 복잡한가’ 같은 여러 요인이 뒤섞여 있습니다.  즉, 모델은 드러나지 않은 주제의 성격과 문장의 복잡도를 동시에 고려해 결정을 내려야 하는, 어떻게 보면 모호한 문제를 풀게 됩니다. 

그렇다면 이 과정을 조금 나누어 설계했다면 어땠을까요?

#### 1단계: 속성을 먼저 분류하기

우선 모델에게 최종 결정을 맡기지 말고, 문장의 속성을 구분하는 과제를 줍니다.  
예를 들어, 모든 문장을 다음 네 가지 케이스 중 하나로 분류하게 하는 것입니다. 

어려운 주제 / 복잡한 문장   -> 라벨 0

어려운 주제 / 간단한 문장   -> 라벨 1

쉬운 주제 / 복잡한 문장      -> 라벨 2

쉬운 주제 / 간단한 문장      -> 라벨 3

이렇게 설계하면 모델은 문장과 라벨 사이의 직접적인 연관성만 학습하면 되기 때문에 훨씬 명확한 신호를 받게 됩니다. 

클래스가 늘어났는데 어째서 문제가 쉬워지는가?

이런 분류 태스크에서는, 클래스의 수보다 라벨에 대한 데이터의 기준이 가지는 직관성, 명확성이 문제의 난이도를 좌우하게 됩니다.  


#### 2단계: 규칙으로 최종 결정하기

이제 모델이 예측한 라벨을 기반으로, 간단한 규칙을 통해 최종 라우팅을 결정하면 됩니다. 라벨 0, 1, 2는 큰 모델로, 3은 작은 모델로 매핑만 해줘도 해결인 것이죠.

이 방식의 장점은 모델은 더 명확하고 일관된 신호를 학습할 수 있다는 것이 있고, 만일 DEFAULT_ROUTER의 정책이 바뀌더라도 모델을 다시 학습시킬 필요가 없다는 것이 있습니다.  2단계의 규칙만 수정하면 시스템 전체를 다르게 작동시킬 수 있습니다. 

컴퓨팅 사고를 알려주는 여러 교육과정에서, 이처럼 복잡한 문제를 조금 더 직관적이고 쉬운 문제로 바꿔가는 것을 문제 분해(Problem Decomposition)라고도 칭합니다.

LM을 활용한 실무에서 여러분은 높은 확률로 제한된 자원만을 활용하게 됩니다. 그리고 제한된 자원 내에서 이러한 문제 분해를 통해 가장 좋은 효율로 문제를 푸는 것은 여러분께 늘 요구되는 최우선의 역량입니다. 

어려운 문제를 만났을 때, 단순히 모델을 키우거나 데이터를 더 모으는 것이 항상 답은 아니라는 것을 알아가실 수 있기를 바랍니다. 

때로는 문제를 더 작은 단위로 나누고, 각 단계의 역할을 명확히 정의하는 것이 훨씬 실용적이고 강력한 접근이 될 수 있습니다.





---


본 과제를 통해 PEFT(LoRA) 기법으로 소형 모델을 최적화하는 엔드투엔드 파이프라인을 구현했습니다. 이 과정에서 데이터 증강 전략의 유효성과 파라미터 효율적 미세조정의 실용성을 확인하며, 기초적인 MLOps 워크플로우를 경험할 수 있었습니다. 여기서 개발된 모델은 자원 분배가 중요한 실제 서비스에 응용될 수 있으며, 향후 더 복잡한 분류 문제로 확장하거나 다양한 모델을 테스트하는 방향으로 발전시킬 수 있습니다. 이번 실습이 최신 AI 기술에 대한 깊이 있는 이해와 실용적 활용 능력을 다지는 계기가 되었기를 바랍니다.